In [2]:
import sys
import os
import logging
import boto3 #aws python sdk
import requests
import json
import base64
import pymysql
import pickle
from datetime import datetime
import pandas as pd


file = open("/Users/kangyeolyoun/Desktop/data_engineering and_pickle file/aws_mysql_db", "rb")
secret_data = pickle.load(file)

host = secret_data[0]
port = int(secret_data[1])
username = "kangyeol"
database = secret_data[2]
password = secret_data[3]
client_id = secret_data[4]
client_secret = secret_data[5]


In [6]:
secret_data[3]

'Ih4312an'

In [ ]:
# aws -> s3 -> create bucket

### python아에서 데이터레이크 구축해보자

In [ ]:
# top_tracks, audio_feature 가져와보자.
# RDS -> artist id가져오고
# Spotify API를 통해서 데이터를 불러오고
# .json file로 저장했다가 S3로 importing 작업

In [4]:
import sys
import os
import logging
import boto3 #aws python sdk
import requests
import json
import base64
import pymysql
import pickle
from datetime import datetime
import pandas as pd


file = open("/Users/kangyeolyoun/Desktop/pickle file/aws_mysql_db", "rb")
secret_data = pickle.load(file)

host = secret_data[0]
port = int(secret_data[1])
username = "kangyeol"
database = secret_data[2]
password = secret_data[3]
client_id = secret_data[4]
client_secret = secret_data[5]

def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers

def main():
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                         port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    # RDS - artist ID load
    cursor.execute("SELECT id FROM artists")
    
    #Top Tracks Spotify (load)
    # list of dictionary 로 변환
    # 파퀫화(spark가 좋아함) (압축해서 비용도 줄고, 퍼포믄서도 증가)
    
#     top_tracks = pd.DataFrame(raw)
#     top_tracks.to_parquet('top-tracks.parquet', engine='pyarrow', compressions='snappy')
    
    dt = datetime.utcnow().strftime("%Y-%m-%d")
    print(dt)
    sys.exit(0)
    

        
    # .json

    with open("top_tracks.json", "w") as f:
        for i in top_tracks:
            json.dump(i, f)
            f.write(os.linesep)
    
    
    
    s3 = boto3.resource("s3")
    #dt를 정의해줌으로써 시간별 변하는 데이터를 저장
    object = s3.Object('spotify--artist', 'dt={}/top-tracks.json'.format(dt))
        
    
if __name__ == "__main__":
    main()

2020-02-11


SystemExit: 0

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
import pyarrow

In [17]:
import sys
import os
import logging
import boto3 #aws python sdk
import requests
import json
import base64
import pymysql
import pickle
from datetime import datetime
import pandas as pd
import jsonpath
import pyarrow

file = open("/Users/kangyeolyoun/Desktop/pickle file/aws_mysql_db", "rb")
secret_data = pickle.load(file)

host = secret_data[0]
port = int(secret_data[1])
username = "kangyeol"
database = secret_data[2]
password = secret_data[3]
client_id = secret_data[4]
client_secret = secret_data[5]

def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers

def main():
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                         port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    # RDS - artist ID load
    cursor.execute("SELECT id FROM artists LIMIT 10")
    
    #jsonpath를 통해, path를 찾고 그 value 반환
    top_track_keys = {
        #track id의 패쓰는 어떻게되?
        "id": "id",   
        "name":"name",
        "popularity":"popularity",
        "external_url": "external_urls.spotify"
    }
    
    #Top Tracks Spotify (load)
    # list of dictionary 로 변환
    # 파퀫화(spark가 좋아함) (압축해서 비용도 줄고, 퍼포믄서도 증가)
    top_tracks = []
    for (id, ) in cursor.fetchall():
        
        URL = "https://api.spotify.com/v1/artists/{}/top-tracks".format(id)
        params = {
            'country':'US'
        }
        r = requests.get(URL, params=params, headers=headers)
        raw = json.loads(r.text)
        
        for i in raw["tracks"]:
            top_track = {}
            for k, v in top_track_keys.items():
                top_track.update({k: jsonpath.jsonpath(i, v)})
                top_track.update({'artist_id': id})
                top_tracks.append(top_track)

    # track_id
    track_ids = [i['id'][0] for i in top_tracks]

    top_tracks = pd.DataFrame(top_tracks)
    top_tracks.to_parquet('top-tracks.parquet', engine='pyarrow', compression='snappy')
    
    
    dt = datetime.utcnow().strftime("%Y-%m-%d")


    s3 = boto3.resource("s3")
    #dt를 정의해줌으로써 시간별 변하는 데이터를 저장
    object = s3.Object('spotify--artist', 'top-tracks/dt={}/top-tracks.parquet'.format(dt))
    data = open('top-tracks.parquet', 'rb')
    object.put(Body=data)
    
    #Audio_feature 은 batch가 가능하므로 배치로 가져온다
    tracks_batch = [track_ids[i: i+100] for i in range(0, len(track_ids), 100)]
    
    
    audio_features = []
    for i in tracks_batch:

        ids = ','.join(i)
        URL = "https://api.spotify.com/v1/audio-features/?ids={}".format(ids)

        r = requests.get(URL, headers=headers)
        raw = json.loads(r.text)

        audio_features.extend(raw['audio_features'])

    audio_features = pd.DataFrame(audio_features)
    audio_features.to_parquet('audio-features.parquet', engine='pyarrow', compression='snappy')

    s3 = boto3.resource('s3')
    object = s3.Object('spotify--artist', 'audio-features/dt={}/top-tracks.parquet'.format(dt))
    data = open('audio-features.parquet', 'rb')
    object.put(Body=data)
        
    
if __name__ == "__main__":
    main()

### s3에 artist_id 저장;;

In [6]:
import sys
import os
import logging
import boto3 #aws python sdk
import requests
import json
import base64
import pymysql
import pickle
from datetime import datetime
import pandas as pd
import jsonpath
import pyarrow

file = open("/Users/kangyeolyoun/Desktop/data_engineering and_pickle file/aws_mysql_db", "rb")
secret_data = pickle.load(file)

host = secret_data[0]
port = int(secret_data[1])
username = "kangyeol"
database = secret_data[2]
password = secret_data[3]
client_id = secret_data[4]
client_secret = secret_data[5]

def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers

def main():
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                         port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
    
    
    # RDS - artist ID load
    cursor.execute("SELECT *FROM artists")
    colnames = [d[0] for d in cursor.description]
    artists = [dict(zip(colnames, row)) for row in cursor.fetchall()]
    artists = pd.DataFrame(artists)
    
    artists.to_parquet("artists.parquet", engine="pyarrow", compression="snappy")
    
    dt = datetime.utcnow().strftime("%Y-%m-%d")
    
    s3 = boto3.resource('s3')
    object = s3.Object('spotify--artist', 'artists/dt={}/artists.parquet'.format(dt))
    data = open('artists.parquet', 'rb')
    object.put(Body=data)
    
if __name__ =="__main__":
    main()